In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

In [2]:
beer_url = "https://bigworldsmallgirl.com/central-texas-brewery-guide/#:~:text=With%20over%2058%20breweries%20in,love%20about%20our%20craft%20beer"

In [3]:
html = requests.get(beer_url)

In [4]:
soup = BeautifulSoup(html.text, 'html.parser')

In [5]:
brewery_data = soup.find('div', class_= 'entry-content clearfix')
brewery_list = brewery_data.find_all('p')

In [6]:
breweries = []
addresses = []
for brewery in brewery_list:
    if (brewery.a and brewery.a.strong):
        brewery_name = brewery.a.strong.text
        brewery_address = brewery.a.next_sibling
        breweries.append(brewery_name)
        addresses.append(brewery_address)
    elif (brewery.strong and brewery.strong.a):
        brewery_name = brewery.strong.a.text
        brewery_address = brewery.strong.next_sibling
        breweries.append(brewery_name)
        addresses.append(brewery_address)

In [7]:
print(len(breweries))
print(len(addresses))

60
60


In [8]:
addresses

[' 10615 Metric Boulevard, 78758',
 ' 2314 Rutland Drive, 78758',
 ' 3001 Industrial Terrace, 78758',
 ' 7020 Easy Wind Drive, 78752 [K] ',
 ' 6015 Dillard Circle, 78752 [K] [BWSG]',
 ' 10001 Metric Boulevard, 78758 [BWSG]',
 ' 2340 W Braker Lane, 78758 [BWSG]',
 ' 4112 Medical Parkway, 78756 [BWSG]',
 ' 2307 Kramer Lane, 78758',
 ' 2400 Patterson Industrial Drive, 78660 ',
 ' 10420 Metric Boulevard, 78758',
 ' 4729 Burnet Road, 78756 [K] [BWSG]',
 ' 1700 Bryant Dr. #107, Round Rock, 78664 [BWSG]',
 ' 7100 FM 3405,\xa0Liberty Hill, 78642',
 ' 16920 Joe Barbee Drive Bldg 1, Round Rock, 78664',
 ' 2200 S Interstate 35 B1, Round Rock, 78681',
 '11880 Hero Way W Suite 208, Leander, ',
 ' 2800 Hoppe Trail, Round Rock, 78681  [K]',
 ' 13010 W. Parmer Ln. #800, Cedar Park,\xa078613 [K] ',
 ' 2500 NE Inner Loop, Georgetown,\xa078626',
 ' 500 Chaparral Drive,\xa0Liberty Hill, 78641 [BWSG]',
 ' 601 E. Whitestone Blvd. #500, Cedar Park, 78613',
 <a aria-label="Central District Brewing (opens in a

In [9]:
df = pd.DataFrame({'Brewery': breweries, 'Address': addresses})
df.head()

,Brewery,Address
0,4th Tap Brewing Cooperative,"10615 Metric Boulevard, 78758"
1,Adelbert’s Brewery,"2314 Rutland Drive, 78758"
2,Austin Beerworks,"3001 Industrial Terrace, 78758"
3,Black Star Co-Op,"7020 Easy Wind Drive, 78752 [K]"
4,Brewtorium,"6015 Dillard Circle, 78752 [K] [BWSG]"


In [10]:
#(\d{3,5}\w[^),]+)
#(\d{3,5}[^,]+
#(\d{3,5}\s\w[^),]+)
#(\d{3,5}-?\w\s\w[^),]+)

In [11]:
df['Street_Address'] = df['Address'].str.extract('(\d{3,5}\s\w[^),]+)', expand=True)
df['Address'] = df['Address'].str.lstrip()
df.tail()

,Brewery,Address,Street_Address
55,Suds Monkey Brewing,"1032 Canyon Bend Drive, Dripping Springs, 78620",1032 Canyon Bend Drive
56,Twisted X Brewing,"23455 Ranch to Market Road 150, Dripping Sprin...",23455 Ranch to Market Road 150
57,Vista Brewing,"13551 Ranch to Market Road 150, Driftwood, 786...",13551 Ranch to Market Road 150
58,Meanwhile Brewing,"(under construction, anticipated opening late ...",3901 Promontory Point Drive
59,Hitmaker Brewing Company,"(under construction) 11160 Circle Drive, 78736",11160 Circle Drive


In [12]:
df['Address']

0                         10615 Metric Boulevard, 78758
1                             2314 Rutland Drive, 78758
2                        3001 Industrial Terrace, 78758
3                      7020 Easy Wind Drive, 78752 [K] 
4                 6015 Dillard Circle, 78752 [K] [BWSG]
5                  10001 Metric Boulevard, 78758 [BWSG]
6                      2340 W Braker Lane, 78758 [BWSG]
7                    4112 Medical Parkway, 78756 [BWSG]
8                               2307 Kramer Lane, 78758
9               2400 Patterson Industrial Drive, 78660 
10                        10420 Metric Boulevard, 78758
11                   4729 Burnet Road, 78756 [K] [BWSG]
12       1700 Bryant Dr. #107, Round Rock, 78664 [BWSG]
13                    7100 FM 3405, Liberty Hill, 78642
14     16920 Joe Barbee Drive Bldg 1, Round Rock, 78664
15           2200 S Interstate 35 B1, Round Rock, 78681
16                11880 Hero Way W Suite 208, Leander, 
17             2800 Hoppe Trail, Round Rock, 786

In [13]:
df['Zipcode'] = df['Address'].str.extract('(\s7\d\d\d\d)', expand=True)
df.tail()

,Brewery,Address,Street_Address,Zipcode
55,Suds Monkey Brewing,"1032 Canyon Bend Drive, Dripping Springs, 78620",1032 Canyon Bend Drive,78620
56,Twisted X Brewing,"23455 Ranch to Market Road 150, Dripping Sprin...",23455 Ranch to Market Road 150,78620
57,Vista Brewing,"13551 Ranch to Market Road 150, Driftwood, 786...",13551 Ranch to Market Road 150,78619
58,Meanwhile Brewing,"(under construction, anticipated opening late ...",3901 Promontory Point Drive,78744
59,Hitmaker Brewing Company,"(under construction) 11160 Circle Drive, 78736",11160 Circle Drive,78736


In [14]:
df.loc[(df['Brewery'] == 'Central District Brewing'), 'Street_Address']= '417 Red River Street'
df.loc[(df['Brewery'] == 'Central District Brewing'), 'Zipcode']= 78701
df.loc[(df['Brewery'] == 'Orf Brewing'), 'Street_Address']= '4700-F Burle­son Road'
df.loc[(df['Brewery'] == 'Batch Brewhouse'), 'Street_Address']= '3220 Manor Road'
df.loc[(df['Brewery'] == 'Batch Brewhouse'), 'Zipcode']= 78723

df.loc[(df['Brewery'] == 'Humble Pint Brewing Co.'), 'Zipcode'] = 78641

In [15]:
df

,Brewery,Address,Street_Address,Zipcode
0,4th Tap Brewing Cooperative,"10615 Metric Boulevard, 78758",10615 Metric Boulevard,78758
1,Adelbert’s Brewery,"2314 Rutland Drive, 78758",2314 Rutland Drive,78758
2,Austin Beerworks,"3001 Industrial Terrace, 78758",3001 Industrial Terrace,78758
3,Black Star Co-Op,"7020 Easy Wind Drive, 78752 [K]",7020 Easy Wind Drive,78752
4,Brewtorium,"6015 Dillard Circle, 78752 [K] [BWSG]",6015 Dillard Circle,78752
5,Celis Brewing,"10001 Metric Boulevard, 78758 [BWSG]",10001 Metric Boulevard,78758
6,Circle Brewing,"2340 W Braker Lane, 78758 [BWSG]",2340 W Braker Lane,78758
7,Draught House,"4112 Medical Parkway, 78756 [BWSG]",4112 Medical Parkway,78756
8,Hopsquad Brewing,"2307 Kramer Lane, 78758",2307 Kramer Lane,78758
9,Flying Man Brewing,"2400 Patterson Industrial Drive, 78660",2400 Patterson Industrial Drive,78660


In [16]:
df_cleaned = df[['Brewery', 'Street_Address', 'Zipcode']]
#df_cleaned

In [17]:
df_cleaned=df_cleaned.drop([30,36,58,59])
df_cleaned=df_cleaned.reset_index(drop=True)

In [18]:
area = []
area += 12 * ['North']
area += 10 * ['Northern Suburbs']
area += 2 * ['Central/Downtown']
area += 9 * ['South']
area += 11 * ['East']
area += 12 * ['West']
df_cleaned['Austin_Area']=area

In [19]:
df_cleaned

,Brewery,Street_Address,Zipcode,Austin_Area
0,4th Tap Brewing Cooperative,10615 Metric Boulevard,78758,North
1,Adelbert’s Brewery,2314 Rutland Drive,78758,North
2,Austin Beerworks,3001 Industrial Terrace,78758,North
3,Black Star Co-Op,7020 Easy Wind Drive,78752,North
4,Brewtorium,6015 Dillard Circle,78752,North
5,Celis Brewing,10001 Metric Boulevard,78758,North
6,Circle Brewing,2340 W Braker Lane,78758,North
7,Draught House,4112 Medical Parkway,78756,North
8,Hopsquad Brewing,2307 Kramer Lane,78758,North
9,Flying Man Brewing,2400 Patterson Industrial Drive,78660,North


In [20]:
df_cleaned.to_csv('data/brewery_list.csv')